In [0]:
# I have created a volume in Unity Catalog - /Volumes/workspace/default/interview_volume where I uploaded title_basics_2018.csv and title_ratings.csv

#Here's the link to UC volume - https://dbc-3c5ac3a3-4279.cloud.databricks.com/explore/data/volumes/workspace/default/interview_volume?o=3125894731479284

In [0]:
# Now we will read the 2018 basics file in basics_df dataframe by keeping .option as as true
basics_df = spark.read.option("header", True).csv("/Volumes/workspace/default/interview_volume/title_basics_2018.csv")

# Now we will read the ratings file in ratings_df daatframe by keeping .option as as true
ratings_df = spark.read.option("header", True).csv("/Volumes/workspace/default/interview_volume/title_ratings.csv")


According to the provided dataset, how many 2018 films were categorized as a Comedy?

In [0]:
#import pyspark functions used for abive questions.
from pyspark.sql.functions import split, explode, col

# Create dataframe genred_df and add replace column 'genre; using withColumn by using Split genres and explode
genres_df = basics_df.withColumn("genre", explode(split(col("genres"), ",")))

# Apply Filter for Comedy and count
comedy_count = genres_df.filter(col("genre") == "Comedy").count()

print(f"Number of 2018 Comedy films: {comedy_count}")

Number of 2018 Comedy films: 2233


According to the provided dataset, how many 2018 films got a score of 8.0 or higher?  (Note that this will require joining the two datasets together)

In [0]:
# Join datasets basics_df and ratings_df on tconst
joined_df = basics_df.join(ratings_df, on="tconst")

# Cast rating to float
joined_df = joined_df.withColumn("averageRating", col("averageRating").cast("float"))

# Filter for rating ≥ 8.0
high_rated_count = joined_df.filter(col("averageRating") >= 8.0).count()

print(f"Number of 2018 films with rating >= 8.0 is {high_rated_count}")

Number of 2018 films with rating >= 8.0 is 780


What was the best film of 2018?

In [0]:
from pyspark.sql.functions import desc

# Use the above joined dataframe and Cast types
joined_df = joined_df.withColumn("numVotes", col("numVotes").cast("int"))

# Filter popular and sort by ordering it by descending order and filtering column 'numVotes' > 1000, then select columns the topmost value using limit(1)
top_film = (
    joined_df.filter(col("numVotes") > 1000)
    .orderBy(desc("averageRating"))
    .select("primaryTitle", "averageRating", "numVotes")
    .limit(1)
)

top_film.display()

primaryTitle,averageRating,numVotes
Dominion,9.2,1387


Do audiences prefer longer films, or shorter films?  You may choose to simply outline your methodology to approach this problem.

In [0]:
from pyspark.sql.functions import when, avg

# Cast runtime to int
joined_df = joined_df.withColumn("runtimeMinutes", col("runtimeMinutes").cast("int"))

# Categorize film length based on runtimeMinutes <90 as short , between 90 to 120 as medium and above 120 and long
bucketed_df = joined_df.withColumn(
    "lengthCategory",
    when(col("runtimeMinutes") < 90, "Short")
    .when((col("runtimeMinutes") >= 90) & (col("runtimeMinutes") <= 120), "Medium")
    .otherwise("Long")
)

# Aggregate ratings and votes by category using groupBy and agg function
preferences = bucketed_df.groupBy("lengthCategory").agg(
    avg("averageRating").alias("avgRating"),
    avg("numVotes").alias("avgVotes")
)

preferences.display()

lengthCategory,avgRating,avgVotes
Short,6.226473819157528,312.1570853167338
Medium,6.191717287968825,2821.146880176698
Long,6.533105019973293,8726.663242009132
